# Tutorial 3: Un-guided Capsule Reentry

## Import statements

In [1]:
###############################################################################
# IMPORT STATEMENTS ###########################################################
###############################################################################
import numpy as np
import pandas as pd
from tudatpy import elements
from tudatpy.kernel import io
from tudatpy.kernel import constants
from tudatpy.kernel import ephemerides
from tudatpy.kernel import numerical_integrators
from tudatpy.kernel import basic_astrodynamics
from tudatpy.kernel import spice_interface
from tudatpy.kernel import propagators
from tudatpy.kernel import simulation_setup
from tudatpy.kernel import unit_tests

In [2]:
# Load spice kernels.
spice_interface.load_standard_kernels()

# Set simulation start epoch.
simulation_start_epoch = 0.0

# Set numerical integration fixed step size.
fixed_step_size = 1.0

In [3]:
###########################################################################
# CREATE ENVIRONMENT ######################################################
###########################################################################

bodies_to_create = ["Earth"]

body_settings = simulation_setup.get_default_body_settings(bodies_to_create)

body_settings["Earth"].ephemeris_settings = simulation_setup.ConstantEphemerisSettings(
    np.zeros(6), "SSB", "J2000")

body_settings["Earth"].rotation_model_settings.reset_original_frame("J2000")

# Create Earth Object.
bodies = simulation_setup.create_bodies(body_settings)

In [4]:
###########################################################################
# CREATE VEHICLE ##########################################################
###########################################################################

# Create vehicle objects.
bodies["Apollo"] = simulation_setup.Body()

bodies["Apollo"].set_aerodynamic_coefficient_interface(unit_tests.get_apollo_coefficient_interface())

bodies["Apollo"].set_constant_body_mass(5.0E3)

In [5]:
###########################################################################
# FINALIZE BODIES #########################################################
###########################################################################

# Finalize body creation.
simulation_setup.set_global_frame_body_ephemerides(bodies, "SSB", "J2000")

In [6]:
###########################################################################
# CREATE ACCELERATIONS ####################################################
###########################################################################

# Define bodies that are propagated.
bodies_to_propagate = ["Apollo"]

# Define central bodies.
central_bodies = ["Earth"]

# Define accelerations acting on Apollo.
accelerations_of_apollo = dict(Earth=[
    simulation_setup.Acceleration.spherical_harmonic_gravity(4, 0),
    simulation_setup.Acceleration.aerodynamic()
])

# Create global accelerations dictionary.
acceleration_dict = dict(Apollo=accelerations_of_apollo)

# Create acceleration models.
acceleration_models = simulation_setup.create_acceleration_models_dict(
    bodies, acceleration_dict,
    bodies_to_propagate, central_bodies)

# Define constant 30 degree angle of attack.
constant_aoa = np.deg2rad(30.0)
bodies["Apollo"].get_flight_conditions(
).get_aerodynamic_angle_calculator(
).set_orientation_angle_functions(lambda: constant_aoa)

In [7]:
###########################################################################
# CREATE PROPAGATION SETTINGS #############################################
###########################################################################

# Set spherical elements for Apollo and convert to Cartesian.
cartesian_initial_state = elements.spherical2cartesian(
    r=spice_interface.get_average_radius("Earth") + 120.0E3,
    lat=np.deg2rad(0.0),
    lon=np.deg2rad(68.75),
    speed=7.7E3,
    fpa=np.deg2rad(-0.9),
    heading=np.deg2rad(34.37))

# Convert the state to the global (inertial) frame.
earth_rotational_ephemeris = bodies["Earth"].get_rotational_ephemeris()
system_initial_state = ephemerides.transform_state_to_global_frame(
    cartesian_initial_state,
    simulation_start_epoch,
    earth_rotational_ephemeris)

# Define list of dependent variables to save.
# TODO: Revise design of dependent variable saves with Python class layer.
dependent_variables_list = [
    propagators.SingleDependentVariableSaveSettings(
        propagators.PropagationDependentVariables.mach_number_dependent_variable,
        "Apollo"
    ),
    propagators.SingleDependentVariableSaveSettings(
        propagators.PropagationDependentVariables.altitude_dependent_variable,
        "Apollo", "Earth"
    ),
    propagators.SingleAccelerationDependentVariableSaveSettings(
        basic_astrodynamics.AvailableAcceleration.aerodynamic,
        "Apollo", "Earth", 1
    ),
    propagators.SingleDependentVariableSaveSettings(
        propagators.PropagationDependentVariables.aerodynamic_force_coefficients_dependent_variable,
        "Apollo"
    )
]
dependent_variables_to_save = propagators.DependentVariableSaveSettings(
    dependent_variables_list)

# Define termination conditions.
termination_dependent_variable = propagators.SingleDependentVariableSaveSettings(
    propagators.PropagationDependentVariables.altitude_dependent_variable,
    "Apollo", "Earth"
)
termination_settings = propagators.PropagationDependentVariableTerminationSettings(
    dependent_variable_settings=termination_dependent_variable,
    limit_value=25.0E3,
    use_as_lower_limit=True,
    terminate_exactly_on_final_condition=False
)

# Create propagation settings.
propagator_settings = propagators.TranslationalStatePropagatorSettings(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    system_initial_state,
    termination_settings,
    propagators.TranslationalPropagatorType.cowell,
    dependent_variables_to_save
)
integrator_settings = numerical_integrators.IntegratorSettings(
    numerical_integrators.AvailableIntegrators.rungeKutta4,
    simulation_start_epoch,
    fixed_step_size
)

In [8]:
###########################################################################
# PROPAGATE ORBIT #########################################################
###########################################################################

# Create simulation object and propagate dynamics.
dynamics_simulator = propagators.SingleArcDynamicsSimulator(
    bodies, integrator_settings, propagator_settings)

solution = dynamics_simulator.get_equations_of_motion_numerical_solution()
dependent = dynamics_simulator.get_dependent_variable_history()